## Prepare data

In [45]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
import numpy as np
training_device

device(type='cuda')

In [ ]:
# pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" evaluate==0.4 --upgrade

In [59]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

median_map = {
    "A1": 2,
    "A2": 5,
    "B1": 8,
    "B2": 11,
    "C1": 14,
    "C2": 16
}

df = pd.read_csv("data_v2/sample_data_acc_gec.csv")
df = df[df["agreement_percentage"] >= 60]
#df['ef_level'] = df.apply(lambda row: median_map[row['cefr_level']] if pd.isna(row['ef_level']) else row['ef_level'], axis=1)

#df["agreement_percentage"].value_counts()
df.head()

,recordId,gpt4o_judge_score,gpt4o_evaluation,nova_judge_score,nova_evaluation,llama3_judge_score,llama3_evaluation,majority_value,agreement_percentage,writing_id,task_id,ef_level,activity_instructions,student_submission,strength,area_for_improvement,comment,raw_correction,correction
0,CALL0008518,5.0,The student demonstrates advanced control over...,5.0,The student's submission is grammatically accu...,5.0,The student's submission is grammatically accu...,5.0,100.000000,99047bf8-f90c-441c-b144-c5c967ba1bed,41c301bb-948b-4f55-a421-9da9be1aea3e,7.0,Listen again to the flight attendant. Type out...,A flight attendant is responsible for the safe...,['You described the role of a flight attendant...,['Try adding an example to support your points...,You made good use of appropriate vocabulary an...,A flight attendant is responsible for the safe...,<input>A flight attendant is responsible for t...
1,CALL0009642,5.0,The student submission demonstrates advanced g...,1.0,The student submission is generally well-const...,5.0,The student demonstrates strong control over g...,5.0,66.666667,86cedceb-f077-4f06-a4ba-5cb433277597,6bd37619-8520-4e9f-b5e8-4d1b218f8a0d,11.0,Read the blog post. Then write about a city yo...,"Last year I visited Paris, and it was an unfor...",['You described personal experiences and obser...,['Try adding a brief summary before the conclu...,Great job describing your visit to Paris! Your...,"Last year I visited Tokyo, and it was an unfor...","<input>Last year I visited <code a=""NOUN"" c=""T..."
2,CALL0009103,4.0,The student demonstrates strong control over g...,3.0,The student demonstrates fair control over a v...,4.0,The student demonstrates good control over gra...,4.0,66.666667,01d1b3d2-dc1d-4cd4-86bc-d7f99aeddaf2,8f1a0c16-d259-4737-af06-b6b6eb9e9799,13.0,A website has invited you to write your opinio...,I believe that everyone who lives in this worl...,"['Clear opinion with logical argumentation', '...",['Add more specific examples to support argume...,Good job on a clear and structured opinion art...,I believe that everyone who lives in this worl...,<input>I believe that everyone who lives in th...
3,CALL0008310,5.0,The student demonstrates high control over gra...,5.0,The student demonstrates advanced control over...,5.0,The student demonstrates excellent control ove...,5.0,100.000000,fafb7e00-2f10-4181-a5d1-83a91b3194ee,bdd2883c-b71b-4fb7-a44c-a53e7bbf26f4,15.0,Write a comparison-contrast essay about your f...,To create a comparison and contrast between my...,['Your essay effectively compares and contrast...,"[""Try adding a hook in the introduction to gra...",Great job on your comparison essay! Keep using...,To create a comparison and contrast between my...,<input>To create a comparison and contrast bet...
4,CALL0028198,0.0,The student's submission consists of a fragmen...,1.0,The student's submission is incomplete and doe...,0.0,The student did not produce a meaningful respo...,0.0,66.666667,19e802df-05a6-4a2b-9562-d5d3fc6f4e38,919c4050-62f4-4659-92e0-b8a696061e03,7.0,Write a summary of the medical profession as o...,NaN,['Your response shows an appropriate attempt t...,['Add a comparison section to highlight simila...,"You have a good start, add more details and co...",It's hard work.,"<input><code a=""PRON"" c=""It"">Its </code><code ..."


In [60]:
df["task_id"].value_counts()

task_id
2d726061-d067-49a6-a950-5c33482ce17f    170
cfc369d9-74da-4013-be4c-b8148993f658    169
51184f66-e317-457a-bda9-d73a241b7871    168
de7a1159-0c06-49bf-a273-7bdd195d6a32    166
28757296-8797-428f-a562-0b03c304b341    166
                                       ... 
178cfadd-4162-4b5b-aaee-97a1bdcd4d98     17
511ccd73-9db8-45e5-8f85-10e244841ce2     16
3ef52563-7f79-446a-b692-6435408e5461     15
a1471208-6755-4814-aa1a-ab276be4d644     13
d2196935-4153-4f5c-a1d2-dae31174ae55     10
Name: count, Length: 160, dtype: int64

In [61]:
df['text'] = (
    "Prompt Level: " + df['ef_level'].astype(str) +
    " [SEP] Prompt: " + df['activity_instructions'] +
    " [SEP] Response: " + df['student_submission'] +
    " [SEP] Correction: " + df['correction']
)

In [62]:
df["text"][1]

'Prompt Level: 11.0 [SEP] Prompt: Read the blog post. Then write about a city you visited. Type in the input box. Write 90-110 words. Use your own words where possible. [SEP] Response: Last year I visited Paris, and it was an unforgettable experience. The city is famous for its elegance, art, and history. I loved walking along the Seine River and seeing the Eiffel Tower sparkle at night. I spent hours at the Louvre, where I admired the Mona Lisa and many other masterpieces. Montmartre was another highlight—its artistic vibe and the view from the Sacré-Cœur were stunning. I also enjoyed tasting French cuisine, especially croissants and macarons. Paris is a magical city that combines beauty, culture, and charm. I can’t wait to go back one day.\n [SEP] Correction: <input>Last year I visited <code a="NOUN" c="Tokyo">Paris</code>, and it was an unforgettable experience. The city is famous for its <code a="NOUN" c="technology">elegance</code>, art, and history. I loved walking <code a="PREP"

In [63]:
df = df[["text", "task_id", "ef_level", "majority_value"]]
df = df.rename(columns={'majority_value': 'label'})
df.head()

,text,task_id,ef_level,label
0,Prompt Level: 7.0 [SEP] Prompt: Listen again t...,41c301bb-948b-4f55-a421-9da9be1aea3e,7.0,5.0
1,Prompt Level: 11.0 [SEP] Prompt: Read the blog...,6bd37619-8520-4e9f-b5e8-4d1b218f8a0d,11.0,5.0
2,Prompt Level: 13.0 [SEP] Prompt: A website has...,8f1a0c16-d259-4737-af06-b6b6eb9e9799,13.0,4.0
3,Prompt Level: 15.0 [SEP] Prompt: Write a compa...,bdd2883c-b71b-4fb7-a44c-a53e7bbf26f4,15.0,5.0
4,NaN,919c4050-62f4-4659-92e0-b8a696061e03,7.0,0.0


In [64]:
# Apply the mapping to the 'labels' column
#df['label'] = df['label'].map(label_mapping)
df.dropna(subset=['label', 'text'], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,text,task_id,ef_level,label
0,Prompt Level: 7.0 [SEP] Prompt: Listen again t...,41c301bb-948b-4f55-a421-9da9be1aea3e,7.0,5.0
1,Prompt Level: 11.0 [SEP] Prompt: Read the blog...,6bd37619-8520-4e9f-b5e8-4d1b218f8a0d,11.0,5.0
2,Prompt Level: 13.0 [SEP] Prompt: A website has...,8f1a0c16-d259-4737-af06-b6b6eb9e9799,13.0,4.0
3,Prompt Level: 15.0 [SEP] Prompt: Write a compa...,bdd2883c-b71b-4fb7-a44c-a53e7bbf26f4,15.0,5.0
4,Prompt Level: 7.0 [SEP] Prompt: Write an onlin...,6e8793e1-760b-43e0-a8a9-44dc11336782,7.0,2.0


In [65]:
df["label"].value_counts()

label
2.0    2069
4.0    1651
3.0    1541
5.0    1445
1.0     636
0.0     130
Name: count, dtype: int64

In [66]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'task_id', 'ef_level', 'label'],
    num_rows: 7472
})

In [67]:
from datasets import ClassLabel, Value, Sequence
new_features = ds.features.copy()
new_features["label"] = ClassLabel(names=[0, 1, 2, 3, 4, 5])
ds = ds.cast(new_features)

# Step 1: Initial train/test split with stratification
train_test_ds = ds.train_test_split(test_size=0.20, seed=20)

# Step 2: Split the test set into half test, half validation
test_valid_split = train_test_ds['test'].train_test_split(test_size=0.5, seed=20)

# Step 3: Combine everything into a single DatasetDict
dataset = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid_split['train'],    # This becomes the test set
    'validation': test_valid_split['test']  # This becomes the validation set
})
ds

Casting the dataset:   0%|          | 0/7472 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'task_id', 'ef_level', 'label'],
    num_rows: 7472
})

In [69]:
# Verify label distribution
from collections import Counter

print("Train label counts:", Counter(dataset['train']['label']))
print("Test label counts:", Counter(dataset['test']['label']))
print("Validation label counts:", Counter(dataset['validation']['label']))

Train label counts: Counter({2: 1658, 4: 1285, 3: 1248, 5: 1163, 1: 525, 0: 98})
Test label counts: Counter({2: 197, 4: 195, 5: 144, 3: 142, 1: 56, 0: 13})
Validation label counts: Counter({2: 214, 4: 171, 3: 151, 5: 138, 1: 55, 0: 19})


In [70]:
import json

# Fonction utilitaire pour sauvegarder un split en JSONL
def save_split_to_jsonl(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for record in dataset_split:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

# Sauvegarde des trois splits
save_split_to_jsonl(dataset['train'], 'data/train_gec.jsonl')
save_split_to_jsonl(dataset['test'], 'data/test_gec.jsonl')
save_split_to_jsonl(dataset['validation'], 'data/validation_gec.jsonl')

## Roberta SFT

In [71]:
import numpy as np 
import evaluate

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, cohen_kappa_score, classification_report
from scipy.stats import pearsonr

metric = evaluate.load("accuracy")

In [72]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convertir les logits en classes prédictes

    # 🎯 Exactitude (Accuracy)
    accuracy = accuracy_score(labels, predictions)

    # 🎯 Précision, Rappel et F1-score (pondérés)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    # 🎯 Score de Cohen's Kappa (pondéré)
    cohen_kappa = cohen_kappa_score(labels, predictions, weights="quadratic")

    # 🎯 Corrélation de Pearson
    pearson_corr, _ = pearsonr(labels, predictions)  # Retourne (coef, p-valeur), on garde seulement coef

     # 🎯 Classification Report
    class_report = classification_report(labels, predictions, output_dict=True)  # Get a dictionary of the report


    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "cohen_kappa": cohen_kappa,
        "pearson_corr": pearson_corr,
        "classification_report": class_report  # Add classification report to the return
    }

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset

# Charger les fichiers JSONL en DatasetDict
dataset = DatasetDict({
    "train": load_dataset("json", data_files="data/train_gec.jsonl")["train"],
    "test": load_dataset("json", data_files="data/test_gec.jsonl")["train"],
    "validation": load_dataset("json", data_files="data/validation_gec.jsonl")["train"]
})

print(dataset)

In [75]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [76]:
dataset["train"][0]

{'text': 'Prompt Level: 7.0 [SEP] Prompt: What problems can happen on board a ship? What are some solutions? Use your own ideas. Choose from one or two of the options provided and write 80-100 words. Options: general cargo safety and stowage; staffing levels and working conditions; general condition and maintenance of the vessel; safety standards; pollution [SEP] Response: A common problem onboard is related to the safety and stowage of general cargo. If cargo is poorly stowed, it can shift during navigation, causing damage to the vessel or even putting the crew at risk. To avoid this, it is essential to follow a proper stowage plan, use appropriate containment materials, and conduct regular inspections of the holds. Another important issue involves safety standards. Equipment such as life jackets and fire extinguishers must be accessible and within their expiration date. Regular training and emergency drills help keep the crew prepared for any situation.\n\n  [SEP] Correction: <input>

In [77]:
tok_test = tokenizer(dataset["train"][1]["text"], max_length=256, truncation=True)
tok_test

{'input_ids': [0, 35396, 3320, 12183, 35, 262, 4, 288, 646, 3388, 510, 742, 42944, 35, 978, 3116, 110, 3231, 12, 4892, 30999, 4, 9427, 7, 1606, 5, 1575, 5, 2257, 138, 56, 45, 1165, 36, 771, 6222, 2257, 6, 588, 12, 958, 2502, 420, 138, 8, 70, 2539, 6, 1950, 6818, 3748, 518, 238, 8, 67, 304, 103, 9, 110, 308, 2956, 4, 21062, 727, 12, 6115, 1617, 4, 7627, 5, 2788, 7, 244, 47, 4, 646, 3388, 510, 742, 19121, 35, 12191, 2488, 6362, 6006, 50118, 50118, 170, 236, 47, 7, 1606, 55, 1575, 7, 5, 695, 7, 5731, 18044, 2688, 11, 84, 609, 4, 1541, 138, 782, 16487, 13247, 14, 817, 84, 1230, 1074, 3013, 8, 18618, 4, 50118, 50118, 6715, 6, 465, 874, 5, 12628, 2192, 52, 236, 47, 7, 1606, 11, 5, 11554, 1743, 2257, 35, 50118, 12, 83, 12332, 14, 5, 2111, 64, 304, 7, 1349, 11, 697, 5, 18866, 9, 3057, 50118, 12, 32048, 62, 10, 980, 147, 916, 32, 441, 7, 8469, 19, 84, 518, 6, 10, 449, 9433, 9, 7359, 13, 697, 323, 50118, 12, 83, 1914, 2931, 14, 2029, 916, 41, 2319, 2996, 1248, 50118, 50118, 20867, 209, 1575, 6, 

In [78]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=312
    )

In [79]:
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)
tokenized_valid = dataset["validation"].map(tokenize_function, batched=True)

Map:   0%|          | 0/5977 [00:00<?, ? examples/s]

Map:   0%|          | 0/747 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

In [80]:
unique_labels = set(dataset['train']['label'])
num_labels = len(unique_labels)
num_labels

6

In [81]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=num_labels)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.d

In [82]:
args = TrainingArguments(
    output_dir="../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented",
    evaluation_strategy="steps",  # Évaluation aux mêmes intervalles que la sauvegarde
    save_strategy="steps",  # Sauvegarde tous les 500 steps
    save_steps=200,
    eval_steps=200,  # ⚠ IMPORTANT : Évaluation aux mêmes steps
    save_total_limit=4,  # Ne garde que 4 checkpoints max
    learning_rate=1e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine", 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,  
    metric_for_best_model="f1",
    logging_steps=100,
    fp16=True,
)

In [83]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in PromptModelConfig has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Using cuda_amp half precision backend
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/trainer.py:593: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [84]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: ef_level, task_id, text. If ef_level, task_id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5977
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1496
  Number of trainable parameters = 355365894
2026/02/23 12:05:40 INFO alembic.runtime.plugins: setup plugin 

Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: ef_level, task_id, text. If ef_level, task_id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 747
  Batch size = 16
***** Running Evaluation *****
  Num examples = 747
  Batch size = 16
Saving model checkpoint to ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200
Saving model checkpoint to ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200
Configuration saved in ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200/config.json
Model weights saved in ../../../model_saved/roberta-base-ft-acc-writing-gec-task-augmented/checkpoint-200/pytorch_model.bin
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/trainer.py:

In [ ]:
trainer.evaluate()

## Detailed Evaluation

In [ ]:
list_topic = dataset["valid"]["task_id"]
list_t_set = set(list_topic)
unique_t = (list(list_t_set))

list_level = dataset["valid"]["level_title"]
list_l_set = set(list_level)
unique_l = (list(list_l_set))


In [ ]:
list_r = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for t in unique_t:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['task_id'] == t)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "task_id": t,
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r.append(r)

In [ ]:
list_r_level = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for l in unique_l:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['level_title'] == l)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r_level.append(r)


In [ ]:
import pandas as pd
df_eval_results = pd.DataFrame(list_r, columns=["task_id", "level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results.head(n=10)

In [ ]:
df_eval_results.to_csv("result_eval_data_roberta_large_writing_task_acc.csv", index=False)

In [ ]:
import pandas as pd
df_eval_results_level = pd.DataFrame(list_r_level, columns=["level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results_level.head(n=20)

In [ ]:
df_eval_results_level.to_csv("result_eval_data_roberta_large_acc_by_level.csv", index=False)

## Onnx Export

In [ ]:
#!pip install onnxruntime

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoTokenizer
import torch
import os
from onnxruntime.quantization import quantize_dynamic, QuantType

# === CONFIGURATION ===
# Chemin vers ton dossier contenant le .bin et le config.json
#model_dir = "model_saved/roberta-large-ft-acc-writing-task-augmented/checkpoint-1800"
model_dir="/tmp/tmpdxu3_htb"
onnx_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800.onnx"
quantized_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800-quantized.onnx"

# === ÉTAPE 1 : Charger le modèle et tokenizer ===
model = RobertaForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")
model.eval()

# === ÉTAPE 2 : Préparer un input fictif ===
dummy_text = "Texte d'exemple pour conversion ONNX"
inputs = tokenizer(dummy_text, return_tensors="pt", padding="max_length", max_length=256)

# === ÉTAPE 3 : Exporter vers ONNX ===
torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    onnx_model_path,
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size"},
    },
    opset_version=14  # ⬅️ change ici
)

print(f"✅ Modèle exporté en ONNX : {onnx_model_path}")

# === ÉTAPE 4 : Quantization dynamique ===
quantize_dynamic(
    model_input=onnx_model_path,
    model_output=quantized_model_path,
    weight_type=QuantType.QInt8
)

print(f"✅ Modèle quantifié en ONNX : {quantized_model_path}")

In [ ]:
import onnxruntime

onnx_session = onnxruntime.InferenceSession(onnx_model_path)
onnx_session_quant = onnxruntime.InferenceSession(quantized_model_path)

In [ ]:
max_length = 256  # Ajuste selon la taille maximale de ton modèle

# Fonction d'inférence ONNX
def onnx_infer(input_texts, onnx_model):
    inputs = tokenizer(input_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    input_ids = inputs["input_ids"].numpy()
    attention_mask = inputs["attention_mask"].numpy()
    onnx_inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
    onnx_outputs = onnx_model.run(None, onnx_inputs)
    return onnx_outputs[0]

from tqdm import tqdm

def evaluate_with_metrics(dataset, onnx_model, batch_size=16):
    all_logits = []
    all_labels = []

    # tqdm pour afficher la progression sur les batches
    for i in tqdm(range(0, len(dataset), batch_size), desc="Evaluation"):
        batch = dataset[i:i + batch_size]
        texts = batch["text"]
        labels = batch["label"]

        logits = onnx_infer(texts, onnx_model)
        all_logits.extend(logits)
        all_labels.extend(labels)

    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)

    # 🔥 Appliquer compute_metrics
    metrics = compute_metrics((all_logits, all_labels))
    return metrics

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset
# === Lancer l'évaluation ===
results = evaluate_with_metrics(valid_data, onnx_model=onnx_session)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset
# === Lancer l'évaluation ===
results = evaluate_with_metrics(valid_data, onnx_model=onnx_session_quant)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")

In [ ]:
import boto3

# Initialize the S3 client
s3 = boto3.client(
    "s3"
)

# Define your bucket name and desired path in S3
bucket_name = "sagemaker-studio-oxs6vznjds"

s3_key = "writing_task_models/accuracy/model_1800_quantized_roberta_large.onnx"  # Change path as needed
# Upload the ONNX file
bucket_path = "sagemaker-studio-oxs6vznjds"
quantized_model_path = "model_saved/roberta-large-ft-acc-writing-task-1800-quantized.onnx"

s3.upload_file(quantized_model_path, bucket_path, s3_key)

print(f"✅ ONNX model uploaded to s3://{bucket_name}/{s3_key}")

## Load Onnx from s3

In [ ]:
import boto3
import onnxruntime as ort

# Define S3 bucket and model key
bucket_name = 'sagemaker-studio-oxs6vznjds'
model_key = 'writing_task_models/accuracy/model_1800_quantized_roberta_large.onnx'
local_model_path = '/tmp/roberta-large-ft-acc-writing-task-1800-quant.onnx'  # or wherever you want to save temporarily

# Initialize boto3 S3 client
s3 = boto3.client('s3')

# Download the ONNX model from S3 to local path
s3.download_file(bucket_name, model_key, local_model_path)

# Load the ONNX model using onnxruntime
session = ort.InferenceSession(local_model_path)

print("ONNX model loaded successfully.")

In [ ]:
valid_data = ds["validation"]  # Ou "valid" selon ton dataset

In [ ]:
results = evaluate_with_metrics(valid_data, onnx_model=session)
print("🎯 Evaluation Results ONNX :")
for k, v in results.items():
    if k == "classification_report":
        print("\n📋 Classification Report :")
        for label, metrics in v.items():
            print(f"{label}: {metrics}")
    else:
        print(f"{k}: {v}")
